In [1]:
import math, pysal, random, shutil
import numpy as np
import csv
import pandas as pd
import geopandas as gp
import csvkit as ck
import itertools

%pylab inline
#%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
#spatial interaction modelling for the regions created from 2001 ward flow data

from pysal.contrib.spint.gravity import Gravity
from pysal.contrib.spint.gravity import Production
from pysal.contrib.spint.gravity import Attraction
from pysal.contrib.spint.gravity import Doubly

#RegFlows = pd.read_csv('C:/Lorraine/DATA/REGRESS/exploratory_regression/ward01_reg20k_ODvars.csv')
RegFlows = pd.read_csv('C:/Lorraine/DATA/WARD_DATA/2001/OUTPUT_FILES/ward01_reg20k_ODvars_ADDITIONAL.csv')
print RegFlows.head()

RegFlows = RegFlows[RegFlows['origin'] != RegFlows['destination']]
#excluding internal ward flows

flows = RegFlows['flow'].values
Oi = RegFlows['OflowsOut'].values
Dj = RegFlows['DflowsIn'].values
Dij = RegFlows['distance'].values
Origin = RegFlows['origin'].values
Destination = RegFlows['destination'].values

RegFlows['PCO_PCD'] = RegFlows['O_CB_PC'] - RegFlows['D_CB_PC']
RegFlows['PCD_PCO'] = RegFlows['D_CB_PC'] - RegFlows['O_CB_PC']
RegFlows['PCO_PCD'] = RegFlows['PCO_PCD'].abs()
RegFlows['PCD_PCO'] = RegFlows['PCD_PCO'].abs()

print (RegFlows['PCO_PCD']==0).sum()
print (RegFlows['PCD_PCO']==0).sum()

RegFlows['PCO_PCD'] = RegFlows['PCO_PCD'].replace(0, 1)
RegFlows['PCD_PCO'] = RegFlows['PCD_PCO'].replace(0, 1)

print (RegFlows['PCO_PCD']==0).sum()
print (RegFlows['PCD_PCO']==0).sum()

o_vars = np.hstack([RegFlows['O_Jobs_2001'].values.reshape((-1,1)), 
                    RegFlows['O_ECON_ACT_P'].values.reshape((-1,1)),
                    #RegFlows['O_CB_PC'].values.reshape((-1,1)),
                    RegFlows['PCO_PCD'].values.reshape((-1,1)),
                    RegFlows['O_RdLenEC_M'].values.reshape((-1,1)),
                    RegFlows['O_PerH_NoCar'].values.reshape((-1,1)),
                    RegFlows['O_Per_NSEC1-3'].values.reshape((-1,1)),
                    RegFlows['O_MDM_2005'].values.reshape((-1,1)),
                    RegFlows['O_POPDENS_01'].values.reshape((-1,1))])

d_vars = np.hstack([RegFlows['D_Jobs_2001'].values.reshape((-1,1)), 
                    RegFlows['D_ECON_ACT_P'].values.reshape((-1,1)),
                    #RegFlows['D_CB_PC'].values.reshape((-1,1)),
                    RegFlows['PCD_PCO'].values.reshape((-1,1)),
                    RegFlows['D_RdLenEC_M'].values.reshape((-1,1)),
                    RegFlows['D_PerH_NoCar'].values.reshape((-1,1)),
                    RegFlows['D_Per_NSEC1-3'].values.reshape((-1,1)),
                    RegFlows['D_MDM_2005'].values.reshape((-1,1)),
                    RegFlows['D_POPDENS_01'].values.reshape((-1,1))])



gravity = Gravity(flows, Oi, Dj, Dij, 'pow')
print gravity.params
#production = Production(flows, Origin, Dj, Dij, 'exp')
#print production.params[-2:]
#attraction = Attraction(flows, Destination, Oi, Dij, 'exp')
#print attraction.params[-2:]
#doubly = Doubly(flows, Origin, Destination, Dij, 'exp')
#print doubly.params[-1:]

production = Production(flows, Origin, d_vars, Dij, 'pow')
print production.params[-2:]

#attraction = Attraction(flows, Destination, Oi, Dij, 'exp')
attraction = Attraction(flows, Destination, o_vars, Dij, 'pow')
print attraction.params[-2:]

doubly = Doubly(flows, Origin, Destination, Dij, 'pow')
print doubly.params[-1:]

#create a table for the global SIM results
R2, adjR2, SSI, SRMSE, AIC = [], [], [], [], []
model_name = ['grav', 'prod', 'att', 'doub']
col_names = ['R2', 'adjR2', 'AIC', 'SRMSE', 'SSI']
models = [gravity, production, attraction, doubly]
for model in models:
    R2.append(model.pseudoR2)
    adjR2.append(model.adj_pseudoR2)
    SSI.append(model.SSI)
    SRMSE.append(model.SRMSE)
    AIC.append(model.AIC)
cols = {'model_name': model_name,
'R2': R2,
'adjR2': adjR2,
'SSI': SSI,
'SRMSE': SRMSE,
'AIC': AIC }
data = pd.DataFrame(cols).set_index('model_name')
data[col_names]
print data
#data.to_csv('C:/Lorraine/OUTPUTS/SpIntStats/wards2001_20k_spintg.csv')
#data.to_csv('C:/Lorraine/OUTPUTS/SpIntStats/Final/Final_wards01_20k_spintg_pow_CathDiff.csv')
data.to_csv('C:/Lorraine/DATA/WARD_DATA/RESULTS/global_wards01_20k_spintg_pow_ADDITIONAL.csv')


local_attraction = attraction.local()
local_production = production.local()

print local_attraction.keys()
print local_production.keys()

local_attractionSOA = pd.DataFrame({'Destination':np.unique(Destination),
                                    'Jobs':np.round(local_attraction['param0'],4),
                                    'Workers': np.round(local_attraction['param1'],4),
                                    'Perc Cath Diff': np.round(local_attraction['param2'],4),
                                    'Road Dist to Emp Centre': np.round(local_attraction['param3'],4),
                                    'Perc Households No Car': np.round(local_attraction['param4'],4),
                                    'Perc NSEC 1-3': np.round(local_attraction['param5'],4),
                                    'Deprivation Rank': np.round(local_attraction['param6'],4),
                                    'Pop Dens': np.round(local_attraction['param7'],4),
                                    'Distance betas':np.round(local_attraction['param8'],4),
                                    'pseudoR2':np.round(local_attraction['pseudoR2'], 4)})

local_productionSOA = pd.DataFrame({'Destination':np.unique(Origin),
                                    'Jobs':np.round(local_production['param0'],4),
                                    'Workers': np.round(local_production['param1'],4),
                                    'Perc Cath Diff': np.round(local_production['param2'],4),
                                    'Road Dist to Emp Centre': np.round(local_production['param3'],4),
                                    'Perc Households No Car': np.round(local_production['param4'],4),
                                    'Perc NSEC 1-3': np.round(local_production['param5'],4),
                                    'Deprivation Rank': np.round(local_production['param6'],4),
                                    'Pop Dens': np.round(local_production['param7'],4),
                                    'Distance betas': np.round(local_production['param8'],4),
                                    'pseudoR2':np.round(local_production['pseudoR2'], 4)})

#print local_valsSOA.head()
print local_attractionSOA.head()
print local_productionSOA.head()

#local_productionSOA.to_csv('C:/Lorraine/OUTPUTS/SpIntStats/Variables/Wards01spintg_localproduction_pow_OD_Lloydvars_4vars.csv')
#local_attractionSOA.to_csv('C:/Lorraine/OUTPUTS/SpIntStats/Variables/Wards01spintg_localattraction_pow_OD_Lloydvars_4vars.csv')

#local_productionSOA.to_csv('C:/Lorraine/OUTPUTS/SpIntStats/Final/20kwards01spintg_localproduction_pow_Cathdiff.csv')
#local_attractionSOA.to_csv('C:/Lorraine/OUTPUTS/SpIntStats/Final/20kwards01spintg_localattraction_pow_Cathdiff.csv')
local_productionSOA.to_csv('C:/Lorraine/DATA/WARD_DATA/RESULTS/20kwards01spintg_localproduction_pow_Additional.csv')
local_attractionSOA.to_csv('C:/Lorraine/DATA/WARD_DATA/RESULTS/20kwards01spintg_localattraction_pow_Additional.csv')

      

#local_gravity = gravity.local(Origin, np.unique(Origin))
#localvals = pd.DataFrame({'betas': local_gravity['param2'],'Origin':np.unique(Origin),'pseudoR2': local_gravity['pseudoR2']})
#print localvals.head()
#print regions.head()
#regions = regions.reset_index()

#local_vals = pd.merge(localvals, regions[['REG_ID', 'geometry']],
                      #left_on='Origin', right_on='REG_ID')
#local_vals = gp.GeoDataFrame(local_vals)
#reg_local_vals = local_vals.dissolve(by='REG_ID')
#local_vals.to_file('regions2001_20k_SI_betas.shp', driver='ESRI Shapefile')
#print len(local_vals)
      

   Unnamed: 0  origin  destination   flow  OflowsOut  DflowsIn      distance  \
0           0       1            1  22864      39424     29920      0.000000   
1           0       2            1     12      29872     29920  77589.383693   
2           0       3            1    307      28897     29920  43950.764101   
3           0       4            1     27      24473     29920  65370.353584   
4           0       5            1     39      27853     29920  90601.120630   

           origx          origy         destx      ...       D_MDM_2005  \
0  307986.129490  420813.415025  307986.12949      ...           305.14   
1  242589.692587  379059.151744  307986.12949      ...           305.14   
2  331892.235605  383932.962812  307986.12949      ...           305.14   
3  298251.530335  356171.936801  307986.12949      ...           305.14   
4  339727.174848  335954.301790  307986.12949      ...           305.14   

   D_CB_PC  D_ECON_ACT_P  D_Jobs_2011  D_Jobs_2001  D_All persons  \